## Introduction

## clean up the downloaded images

This is too much effort, should have just deleted any failing ones. I'm not sure why the AVIF images wouldn't load.

In [ ]:
from fastai.vision.all import *

In [ ]:
!find bird_cat_dog/ -type f | xargs rename 's/\..*/.jpg/'

In [ ]:
!find bird_cat_dog/ -size 0 | xargs rm -v

removed 'bird_cat_dog/gimg/dog_/3f824c0e-aa4b-4395-acdd-15a4243f0e41.jpg'
removed 'bird_cat_dog/laion/bird_/a9ff3712-5401-49e7-a9e9-1946a6ef46a9.jpg'
removed 'bird_cat_dog/laion/dog_/96b30f11-eb9a-4960-a41c-667bcd3d4b61.jpg'


In [ ]:
failed = verify_images(get_image_files(data))

In [ ]:
import subprocess
with open("failed.txt", "w") as f:
    for p in failed:
        print(p, file=f)

In [ ]:
!< failed.txt xargs file | sed 's/.*: *//' | sed 's/ (.*//' | uniqoc

13	HTML document, ASCII text, with very long lines
6	JavaScript source, ASCII text, with very long lines
3	HTML document, ASCII text
1	HTML document, Unicode text, UTF-8 text, with CRLF, LF line terminators
7	HTML document, Unicode text, UTF-8 text, with very long lines
2	ISO Media, AVIF Image
1	gzip compressed data, from Unix, original size modulo 2^32 98001
2	HTML document, Unicode text, UTF-8 text


In [ ]:
!< failed.txt xargs file | grep AVIF

bird_cat_dog/gimg/bird_cat_/7ad0c40a-afaa-4b63-938d-8d661a5c2f7f.jpg:     ISO Media, AVIF Image
bird_cat_dog/gimg/cat_dog_/cac07371-a384-4849-ad10-b7d143fcc4d2.jpg:      ISO Media, AVIF Image


In [ ]:
!< failed.txt xargs file | grep AVIF | cut -d: -f1 | xa mogrify

In [ ]:
len(failed)

35

In [ ]:
failed = verify_images(get_image_files(data))

In [ ]:
len(failed)

33

In [ ]:
!< failed.txt xargs file | grep gzip | cut -d: -f1 | xa rename -v 's/$/.gz/'

bird_cat_dog/laion/bird_/d9401f3b-c20e-4607-aa03-5a6049d714f5.jpg renamed as bird_cat_dog/laion/bird_/d9401f3b-c20e-4607-aa03-5a6049d714f5.jpg.gz


In [ ]:
!gunzip bird_cat_dog/laion/bird_/d9401f3b-c20e-4607-aa03-5a6049d714f5.jpg.gz

In [ ]:
failed = verify_images(get_image_files(data))

In [ ]:
len(failed)

33

In [ ]:
failed.map(Path.unlink)

(#33) [None,None,None,None,None,None,None,None,None,None...]

In [ ]:
!find bird_cat_dog/ -type f | xa file | perl -pe 's/^.*?: *//; s/,.*//' | uniqoc

3147	JPEG image data
294	PNG image data
467	RIFF (little-endian) data
15	GIF image data
50	ISO Media


Let's rename the images again to give them the right file extensions.

In [ ]:
!find bird_cat_dog/ -type f | xa file | head -n 2

bird_cat_dog/ddg/bird_/4034a955-d434-47b1-874e-10dd77414160.jpg:         JPEG image data, JFIF standard 1.01, aspect ratio, density 1x1, segment length 16, progressive, precision 8, 1920x1271, components 3
bird_cat_dog/ddg/bird_/f3ae56d2-ebbe-4f09-b435-1d60f61f4e6a.jpg:         PNG image data, 500 x 401, 8-bit/color RGBA, interlaced


In [ ]:
!find bird_cat_dog/ -type f | xa file | perl -ne '/(.*?)(\.\w+):\s*(\w+)/; print "$1 $2 ", lc $3, "\n"' | head -n2

bird_cat_dog/ddg/bird_/4034a955-d434-47b1-874e-10dd77414160 .jpg jpeg
bird_cat_dog/ddg/bird_/f3ae56d2-ebbe-4f09-b435-1d60f61f4e6a .jpg png
Unable to flush stdout: Broken pipe


In [ ]:
images = get_image_files(data)
len(images)

3973

In [ ]:
!find bird_cat_dog/ -type f | xa file | perl -ne '/(.*?)(\.\w+):\s*(\w+)/; rename("$1$2", "$1.".lc($3))'

In [ ]:
images = get_image_files(data)
len(images)

3456

In [ ]:
get_image_files??

Signature: get_image_files(path, recurse=True, folders=None)
Source:   
def get_image_files(path, recurse=True, folders=None):
    "Get image files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=image_extensions, recurse=recurse, folders=folders)
File:      /opt/venvs/python3.10-ai/venv/lib/python3.10/site-packages/fastai/data/transforms.py
Type:      function

In [ ]:
{'.jpeg','.png','.riff','.gif','.iso'} - image_extensions

{'.iso', '.riff'}

In [ ]:
!find bird_cat_dog/ -type f | xargs rename 's/\.riff$/.webp/; s/\.iso$/.avif/'

In [ ]:
images = get_image_files(data)
len(images)

3973

In [ ]:
mv -v bird_cat_dog bird_cat_dog.1.orig

renamed 'bird_cat_dog' -> 'bird_cat_dog.orig'


## resize the images to not larger than 400x400

In [ ]:
resize_images('bird_cat_dog.1.orig', dest='bird_cat_dog.2.resized', max_size=400, recurse=True)

In [ ]:
!du -csh bird_cat_dog*

2.0G	bird_cat_dog.1.orig
101M	bird_cat_dog.2.resized
2.1G	total


## combine images from the different search engines together

In [ ]:
cd ~/ai/blog/posts/multilabel

/home/sam/ai/blog/posts/multilabel


In [ ]:
ls bird_cat_dog.2.resized/

ddg/  gimg/  laion/


In [ ]:
!cp -al bird_cat_dog.2.resized bird_cat_dog.3.together

In [ ]:
cd bird_cat_dog.3.together/

/home/sam/ai/blog/posts/multilabel/bird_cat_dog.3.together


In [ ]:
!(mkdir -p $(cd ddg; ls); rename 's{(\w+)/(\w+)/(.*)}{$2/${1}_$3}' */*/*)

In [ ]:
!find . -mindepth 1 -depth -type d | xa rmdir --ignore-fail-on-non-empty

In [ ]:
cd ..

/home/sam/ai/blog/posts/multilabel
